# Note
In the few upcoming notebooks we will understand the basic machine learning workflow which consists of:
    1. Working with Data - Tensorsm, Datasets, DataLoaders and Transforms
    2. Creating Models - Building the neural network, understanding the automatic differentiation using torch.autograd
    3. Optimizing Model Parameters
    4. Save and Load Model.

All the scripts will be written in .ipynb files and using pytorch api.

Dataset used: FashionMNIST
Tutorial Link: https://pytorch.org/tutorials/beginner/basics/intro.html

1. Working with data

PyTorch has two methods to work with data: 'torch.utils.data.DataLoader' and 'torch.utils.data.Dataset'. Dataset stores the samples and labels while DataLoader is an iterable object which wraps about the Dataset object.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Every TorchVision "Dataset" includes two arguments - "transform" and "target_transform" - to modify the samples and labels respectively

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


Now, we pass "Dataset" as an argument to "DataLoader". This wraps an iterable over our FashionMNIST dataset and supports the following operation:

    a) automatic batching 

    b) Sampling 

    c) shuffling 

    d) multiprocess data loading 

In [3]:
batch_size = 64

In [4]:
# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 2. Creating Models